# PyMart - A Python Shopping System

In [7]:
import datetime

# Decorator for logging
def log_action(func):
    def wrapper(*args, **kwargs):
        print(f"[LOG] Calling {func.__name__} at {datetime.datetime.now()}")
        return func(*args, **kwargs)
    return wrapper


class Product:
    """Basic Product class"""
    def __init__(self, name, price, stock=10):
        self.name = name
        self.price = price
        self._stock = stock  # encapsulation (private-like)

    def __str__(self):
        return f"{self.name.capitalize()} : ₹{self.price} (Stock: {self._stock})"

    def __len__(self):
        return self._stock

    def reduce_stock(self, qty):
        if qty <= self._stock:
            self._stock -= qty
        else:
            raise ValueError("Not enough stock!")

    @classmethod
    def from_string(cls, data_str):
        """Create product from 'name,price,stock' string"""
        name, price, stock = data_str.split(",")
        return cls(name, int(price), int(stock))

    @staticmethod
    def tax(amount):
        """Static method for tax calculation"""
        return amount * 0.05


class DiscountedProduct(Product):
    """Inheritance + Polymorphism"""
    def __init__(self, name, price, stock=10, discount=0.1):
        super().__init__(name, price, stock)
        self.discount = discount

    def __str__(self):
        return f"{self.name.capitalize()} : ₹{self.price} (-{int(self.discount*100)}% off)"

    def final_price(self):
        return self.price * (1 - self.discount)


class Cart:
    def __init__(self):
        self.items = {}

    @log_action
    def add_item(self, product, qty):
        product.reduce_stock(qty)
        self.items[product] = self.items.get(product, 0) + qty

    @log_action
    def remove_item(self, product, qty):
        if product in self.items:
            self.items[product] -= qty
            if self.items[product] <= 0:
                del self.items[product]

    def __iter__(self):
        """Generator for iterating cart items with discount handling"""
        for product, qty in self.items.items():
            if isinstance(product, DiscountedProduct):
                original = product.price * qty
                discounted = product.final_price() * qty
                yield (f"{product.name} x{qty} = ₹{original:.2f} "
                       f"(Discounted: ₹{discounted:.2f}, Saved: ₹{original - discounted:.2f})")
            else:
                yield f"{product.name} x{qty} = ₹{product.price * qty:.2f}"

    def checkout(self):
        total = sum(
            (p.final_price() if isinstance(p, DiscountedProduct) else p.price) * qty
            for p, qty in self.items.items()
        )
        total += Product.tax(total)
        return total


class Shop:
    def __init__(self):
        self.products = [
            Product("apple", 30, 20),
            Product("banana", 20, 15),
            DiscountedProduct("milk", 50, 10, discount=0.2),
            Product("bread", 40, 25),
            Product.from_string("eggs,60,12")  # classmethod usage
        ]
        self.cart = Cart()

    def show_products(self):
        print("\n--- Available Products ---")
        [print(p) for p in self.products]  # list comprehension

    def find_product(self, name):
        return next((p for p in self.products if p.name == name.lower()), None)

    def run(self):
        while True:
            print("\nOptions: show, add, remove, cart, checkout, exit")
            choice = input("Enter choice: ").strip().lower()

            try:
                if choice == "show":
                    self.show_products()
                elif choice == "add":
                    name = input("Enter product name: ").strip().lower()
                    qty = int(input("Enter quantity: "))
                    product = self.find_product(name)
                    if product:
                        self.cart.add_item(product, qty)
                    else:
                        print("Product not found.")
                elif choice == "remove":
                    name = input("Enter product name: ").strip().lower()
                    qty = int(input("Enter quantity to remove: "))
                    product = self.find_product(name)
                    if product:
                        self.cart.remove_item(product, qty)
                    else:
                        print("Product not found.")
                elif choice == "cart":
                    print("\n--- Your Cart ---")
                    for line in self.cart:  # generator iteration
                        print(line)
                elif choice == "checkout":
                    total = self.cart.checkout()
                    print(f"\nFinal Bill: ₹{total:.2f}")
                    # ✅ UTF-8 safe file writing
                    with open("receipt.txt", "w", encoding="utf-8") as f:
                        f.write("Receipt\n")
                        for line in self.cart:
                            f.write(line + "\n")
                        f.write(f"Total: ₹{total:.2f}\n")
                    print("Receipt saved to receipt.txt")
                    print("\n Thank you for shopping with us! We hope to see you again.")
                    break
                elif choice == "exit":
                    print("Exiting shop. Goodbye!")
                    break
                else:
                    print("Invalid choice. Try again.")
            except Exception as e:
                print(f"Error: {e}")


# Run the Shop

if __name__ == "__main__":
    shop = Shop()
    shop.run()



Options: show, add, remove, cart, checkout, exit


Enter choice:  show



--- Available Products ---
Apple : ₹30 (Stock: 20)
Banana : ₹20 (Stock: 15)
Milk : ₹50 (-20% off)
Bread : ₹40 (Stock: 25)
Eggs : ₹60 (Stock: 12)

Options: show, add, remove, cart, checkout, exit


Enter choice:  milk


Invalid choice. Try again.

Options: show, add, remove, cart, checkout, exit


...

[LOG] Calling add_item at 2025-11-11 20:49:32.947146

Options: show, add, remove, cart, checkout, exit


Enter choice:  cart



--- Your Cart ---
milk x4 = ₹200.00 (Discounted: ₹160.00, Saved: ₹40.00)

Options: show, add, remove, cart, checkout, exit


Enter choice:  checkout



Final Bill: ₹168.00
Receipt saved to receipt.txt

 Thank you for shopping with us! We hope to see you again.
